In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
!pip install pymorphy2
import pymorphy2
from tqdm import tqdm_notebook

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# выгружаем файлы

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
# создаём train и test

x_train = train_df["title"].values
x_test = test_df["title"].values

In [ ]:
# токенизация

def Tokenizer(text):
    return re.findall("[а-яА-ЯёЁa-zA-Z\d\+]+", text.lower())

x_tokenized = [Tokenizer(text) for text in x_train]
x_test_tokenized = [Tokenizer(text) for text in x_test]

In [ ]:
# удаляем выбросы (слишком короткие и слишком длинные запросы)

def find_title_length(titles):
  title_length_dict = {}
  for i in range(len(titles)):
    title_length_dict[i] = len(titles[i])
  return title_length_dict

title_lenght = find_title_length(x_tokenized)

x_tokenized_cleaned = {}
for i, title in enumerate(x_tokenized):
  x_tokenized_cleaned[i] = title

train_df_cleaned = train_df.copy()
for i in range(len(x_tokenized)):
  if (title_lenght[i] < 2) or (title_lenght[i] > 22):
    train_df_cleaned.drop(i, inplace=True)
    if i in x_tokenized_cleaned:
      del x_tokenized_cleaned[i]

train_df_cleaned = train_df_cleaned.reset_index(drop=True)
x_tokenized = list(x_tokenized_cleaned.values())

In [ ]:
# создаём список стоп-слов

stopword_set1 = set(nltk.corpus.stopwords.words('russian'))
stopword_set2 = set(nltk.corpus.stopwords.words('english'))
stopword_set = stopword_set1.union(stopword_set2)

In [ ]:
# лемматизация русских слов

lemmatizer = pymorphy2.MorphAnalyzer()
lemmatizer_cache = {}

def lemmatize(token):
  if lemmatizer.word_is_known(token):
    if token not in lemmatizer_cache:
      lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
    return lemmatizer_cache[token]
  return token

x_lemmatized_ru = [[lemmatize(token) for token in text] for text in tqdm_notebook(x_tokenized)]
x_test_lemmatized_ru = [[lemmatize(token) for token in text] for text in tqdm_notebook(x_test_tokenized)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
# лемматизация английских слов + чистим стоп-слова

lemmatizer = WordNetLemmatizer()
lemmatizer_cache = {}

def lemmatize(token):
  if token not in lemmatizer_cache:
    lemmatizer_cache[token] = lemmatizer.lemmatize(token)
  return lemmatizer_cache[token]

x_lemmatized = [[lemmatize(token) for token in text] for text in tqdm_notebook(x_lemmatized_ru)]
x_cleared = [[token for token in text if token not in stopword_set] for text in x_lemmatized]
x_test_lemmatized = [[lemmatize(token) for token in text] for text in tqdm_notebook(x_test_lemmatized_ru)]
x_test_cleared = [[token for token in text if token not in stopword_set] for text in x_test_lemmatized]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [ ]:
# собираем списки слов назад в тексты

for i in tqdm_notebook(range(len(x_cleared))):
  train_df_cleaned['title'].loc[i] = ' '.join(x_cleared[i])

for i in tqdm_notebook(range(len(x_test_cleared))):
  test_df['title'].loc[i] = ' '.join(x_test_cleared[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion

In [ ]:
# удаляем домены из url

for i in tqdm_notebook(range(len(train_df_cleaned["url"]))):
  k = train_df_cleaned["url"].loc[i].rsplit('.', 1)
  train_df_cleaned["url"].loc[i] = k[0]

for i in tqdm_notebook(range(len(test_df["url"]))):
  k = test_df["url"].loc[i].rsplit('.', 1)
  test_df["url"].loc[i] = k[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [ ]:
# чистим url

for i in tqdm_notebook(range(len(train_df_cleaned["url"]))):
  train_df_cleaned["url"].loc[i] = re.sub("www|\W", '', train_df_cleaned["url"].loc[i])

for i in tqdm_notebook(range(len(test_df["url"]))):
  test_df["url"].loc[i] = re.sub("www|\W", '', test_df["url"].loc[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
y_train = train_df_cleaned["target"].astype(int).values
df_train = train_df_cleaned.drop('target', axis=1)

In [ ]:
from sklearn.base import TransformerMixin


class ColumnExtractor(TransformerMixin):
    
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.column_name].values

In [ ]:
pipeline = Pipeline([
    (
        'features', 
        FeatureUnion([
            (
                'title', 
                Pipeline([
                    ('extractor', ColumnExtractor('title')),
                    (
                        'vectorizer', 
                        TfidfVectorizer(lowercase=True, ngram_range=(1, 5), min_df=10, max_df=0.6)
                    )
                ])
            ),
            (
                'url',
                Pipeline([
                    ('extractor', ColumnExtractor('url')),
                    (
                        'vectorizer', 
                        TfidfVectorizer(lowercase=True, ngram_range=(2, 5), analyzer='char', min_df=10, max_df=0.6)
                    )
                ])
            )
        ])
    ),
    ('clf', LogisticRegression(class_weight='balanced'))
])

In [ ]:
pipeline.fit(df_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('title',
                                                 Pipeline(memory=None,
                                                          steps=[('extractor',
                                                                  <__main__.ColumnExtractor object at 0x7f2d7908e320>),
                                                                 ('vectorizer',
                                                                  TfidfVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.float64'>,
                                                                                  enc

In [ ]:
test_df["target"] = pipeline.predict(test_df).astype(bool)
test_df[["id", "target"]].to_csv("porn_predicted_2.csv", index=False)
!cat porn_predicted_2.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
